Использовать Promt Engineering техники (Few Shot и т.п.) + Agno Guardrails для ограничения ввода пользователя и вывода модели в своём кастомном примере 

In [9]:
from agno.agent import Agent
from agno.guardrails import BaseGuardrail
from agno.exceptions import InputCheckError, OutputCheckError
from dotenv import load_dotenv
import os

load_dotenv()

assert os.getenv("OPENAI_API_KEY"), "OPENAI_API_KEY not loaded"

import os
print(os.getenv("OPENAI_API_KEY")[:8], "OK")


AssertionError: OPENAI_API_KEY not loaded

In [3]:
few_shot_examples = """
Ты прям яро русский человек, который слушает шамана, и любит путешествовать по Сибири. Но некоторые слова тебя тригерят.
"""

In [4]:
class InputGuardrail(BaseGuardrail):
    def check(self, run_input):
        tekst = run_input.input_content_string().lower()
        plokhie_slova = ["антигойда", "америка", "антисво", "сша", "русня"]
        for slovo in plokhie_slova:
            if slovo in tekst:
                raise InputCheckError(slovo)
        if len(tekst) > 500:
            raise InputCheckError("чёт ты много пиздишь")
        if len(tekst) < 3:
            raise InputCheckError("чёт ты мало пиздишь")
    async def async_check(self, run_input):
        return self.check(run_input)


In [5]:
class OutputGuardrail(BaseGuardrail):
    def check(self, run_output):
        tekst = str(run_output.content).lower()
        if len(tekst) < 10:
            raise OutputCheckError("Чёт модель мало выдала")
        if any(slovo in tekst for slovo in ["антигойда", "америка", "сша"]):
            raise OutputCheckError("В выводе опасный контент")
    async def async_check(self, run_output):
        return self.check(run_output)

In [6]:
def sozdat_goida_bota():
    agent = Agent(
        name="GoidaBot",
        model="openai:gpt-4o-mini",
        instructions=f"""{few_shot_examples}
Я русский
Я вдыхаю этот воздух
Солнце в небе смотрит на меня
Надо мной летает вольный ветер
Он такой же, как и я
И хочется просто любить и дышать
И мне другого не нужно
Такой, какой есть, и меня не сломать
И всё потому что
Я русский, я иду до конца
Я русский, моя кровь от отца, xeй
Я русский, и мне повезло
Я русский всему миру назло
Я русский
В небо улетает эта песня
И зовёт меня с собой
А во мне пылает моё сердце
Освещая путь домой
Где хочется просто любить и дышать
И мне другого не нужно
Такой уж я есть, и меня не сломать
И всё потому что
Я русский, я иду до конца
Я русский, моя кровь от отца, xeй
Я русский, и мне повезло
Я русский всему миру назло
Я русский
Я русский, я иду до конца
Я русский, моя кровь от отца
Я русский, и мне повезло
Я русский всему миру назло
Я русский
""",
        markdown=False,
        pre_hooks=[InputGuardrail()],
        post_hooks=[OutputGuardrail()]
    )
    return agent


In [7]:
bot = sozdat_goida_bota()
try:
    otvet = bot.run("ГООООООООООЙДА, ZOV ZOV")
    print(otvet.content)
except Exception as e:
    print(e)

ERROR    OPENAI_API_KEY not set. Please set the OPENAI_API_KEY environment variable.

ERROR    Error from OpenAI API: The api_key client option must be set either by passing api_key to the client or by
         setting the OPENAI_API_KEY environment variable

WARNING  Attempt 1/1 failed: The api_key client option must be set either by passing api_key to the client or by   
         setting the OPENAI_API_KEY environment variable

ERROR    Failed after 1 attempts. Last error using OpenAIChat(gpt-4o-mini)

The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable
